# FHIR Bundles from MDH `fhirbulk` Table

This query combines all available resources for each participant and returns a row per participant with a consolidated FHIR bundle.

In [ ]:
query_string = """
    with resources_with_patient_reference as (
            select
                concat('Patient/', id) as patient_reference,
                participantidentifier,
                resource,
                row_number() over (partition by participantidentifier, providername order by pim._provenance['exportstartdate'] desc) as rn
            from
                fhirbulk
                join fhirbulkparticipantidentifiersmapping pim on pim.fhirpatientid = fhirbulk.id
            where
                resourcetype = 'Patient'
        ), 
        by_patient_reference as (
        select
            patient_reference,
            max(participantidentifier) as participantidentifier,
            array_agg(concat('{"resource":', resource, '}')) resource_array
        from
            resources_with_patient_reference
        where 
            rn = 1
        group by 1
        )
        select
            patient_reference,
            participantidentifier,
            concat(
                '{',
                '"resourceType":"Bundle", ',
                concat('"entry":[', array_join(resource_array, ','), '], '),
                concat('"total":', cast(cardinality(resource_array) as varchar(1000))),
                '}'
            ) as fhir_bundle
        from
            by_patient_reference
        where 
            patient_reference is not null
"""

In [ ]:
import os, sys; sys.path.append("..") 
from src.mdh_query import MDHQuery

import pandas as pd

mdh_query = MDHQuery(
    project_schema_name = "",
    athena_workgroup = "",
    athena_output_bucket_location = "",
    aws_profile_name = "" 
)
mdh_query.get_query_result(query_string)